In [ ]:
# train chatbot

import random
import json
import pickle
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
# work working worked works == same word 이걸 해주는게 WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(documents)

words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))

classes=sorted(set(classes))

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

training=[]
output_empty=[0]*len(classes)

for document in documents:
  bag=[]
  word_patterns=document[0]
  word_patterns=[lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  output_row=list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag, output_row])

random.shuffle(training)
training=np.array(training)

train_x=list(training[:,0])
train_y=list(training[:,1])

model=Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

sgd=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

hist=model.fit(np.array(train_x),np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbotmodel.h5',hist)
print("Done")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[(['hello'], 'greetings'), (['hey'], 'greetings'), (['hi'], 'greetings'), (['good', 'day'], 'greetings'), (['Greetings'], 'greetings'), (['what', "'s", 'up', '?'], 'greetings'), (['how', 'is', 'it', 'going', '?'], 'greetings'), (['cya'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['bye'], 'goodbye'), (['cao'], 'goodbye'), (['see', 'ya'], 'goodbye'), (['how', 'old'], 'age'), (['how', 'old', 'is', 'florian'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['how', 'old', 'are', 'you'], 'age'), (['age', '?'], 'age'), (['what', 'is', 'your', 'name'], 'name'), (['what', 'should', 'I', 'call', 'you'], 'name'), (['whats', 'your', 'name', '?'], 'name'), (['who', 'are', 'y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
8/8 [==============================] - 1s 4ms/step - loss: 1.9735 - accuracy: 0.1389
Epoch 2/200
8/8 [==============================] - 0s 2ms/step - loss: 1.9311 - accuracy: 0.1944
Epoch 3/200
8/8 [==============================] - 0s 3ms/step - loss: 1.9163 - accuracy: 0.1944
Epoch 4/200
8/8 [==============================] - 0s 3ms/step - loss: 1.8969 - accuracy: 0.2222
Epoch 5/200
8/8 [==============================] - 0s 3ms/step - loss: 1.8309 - accuracy: 0.2778
Epoch 6/200
8/8 [==============================] - 0s 2ms/step - loss: 1.6901 - accuracy: 0.3889
Epoch 7/200
8/8 [==============================] - 0s 2ms/step - loss: 1.6491 - accuracy: 0.3611
Epoch 8/200
8/8 [==============================] - 0s 3ms/step - loss: 1.5876 - accuracy: 0.5000
Epoch 9/200
8/8 [==============================] - 0s 3ms/step - loss: 1.5828 - accuracy: 0.3333
Epoch 10/200
8/8 [==============================] - 0s 3ms/step - loss: 1.5340 - accuracy: 0.4722
Epoch 11/200
8/8 [===========

In [ ]:
# build chatbot

import random
import json
import pickle
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

lemmatizer = WordNetLemmatizer()
intents=json.loads(open('intents.json').read())

words=pickle.load(open('words.pkl','rb'))
classes=pickle.load(open('classes.pkl','rb'))
model=load_model('chatbotmodel.h5')

def clean_up_sentence(sentence):
  sentence_words=nltk.word_tokenize(sentence)
  sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words=clean_up_sentence(sentence)
  bag=[0]*len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word==w: #else: 0
        bag[i]=1
  return np.array(bag)

def predict_class(sentence):
  bow=bag_of_words(sentence)
  res=model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD=0.25
  results=[[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

  results.sort(key=lambda x: x[1], reverse=True)
  return_list=[]
  for r in results:
    return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
  return return_list

def get_response(intents_list,intents_json):
  tag = intents_list[0]['intent']
  list_of_intents=intents_json['intents']
  for i in list_of_intents:
    if i['tag']==tag:
      result=random.choice(i['responses'])
      break
  return result

print("GO! Bot is running!")

while True:
  message=input("")
  ints=predict_class(message)
  res=get_response(ints,intents)
  print(res)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
GO! Bot is running!
hey
Hello!
hi
Hello!
how are you?
22years!
how are you?
22years!
what is your name my friend!!
You can call me Neural!.
what are my stoks doing?
You own the following shares: ABBV, AAPL, FB, NVDA and an ETF of the S&P 500 Index!
goodbye
Talk to you later


KeyboardInterrupt: ignored